In [ ]:
import pandas as pd
import json
import glob
import numpy as np


In [ ]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)

dat = []

files = glob.glob("data/pois/*.json")
for infile in files:
    with open(infile) as json_file:
        d=json.load(json_file)

        for p in d['results']:
            if 'synagogue' in p['types']:
                datum = {}
                for f in ['name', 'place_id', 'vicinity']:
                    datum[f] = p.get(f)
                for f in ['lat', 'lng']:
                    datum[f] = p['geometry']['location'][f]
                datum['link'] = make_clickable('https://www.google.com/maps/place/?q=place_id:' + datum['place_id'], datum['place_id'])
                dat.append(datum)

In [ ]:
df = pd.DataFrame(dat).drop_duplicates(ignore_index = True)
df.to_csv('data/synagogues.csv')

In [ ]:
from shapely.geometry import Point, Polygon

# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

# Create a Polygon
coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]
poly = Polygon(coords)

# Let's check what we have
print(p1)

print(p2)

print(poly)


# Check if p1 is within the polygon using the within function
print(p1.within(poly))

# Check if p2 is within the polygon
print(p2.within(poly))



In [ ]:
def read_borders(infile):
    with open(infile) as json_file:
        d=json.load(json_file)

    coords = d['features'][0]['geometry']['coordinates'][0][0]
    # swap coordinates so it's (lat, lng)
    for i in range(len(coords)):
        coords[i] = (coords[i][1], coords[i][0])
    return Polygon(coords)


In [ ]:
borders = read_borders('data/gadm41_ISR_0.json')
(minx, miny, maxx, maxy) = borders.bounds

pt = Point(29.751355, 34.373654) # Al temd, Sinai
print(pt.within(borders))
pt = Point(29.968069, 34.778875 ) # somewhere in Eilot
print(pt.within(borders))
pt = Point(30.184895718556003, 34.66956055958991) # somewhere in Egypt
print(pt.within(borders))
pt = Point(30.79287035913899, 34.47324188156933) # ezuz
print(pt.within(borders))

In [ ]:
hits0only = []
hits1only = []
borders0 = read_borders('data/gadm41_ISR_0.json')
borders1 = read_borders('data/gadm41_ISR_1.json')

for _ in range(50000):
    x = np.random.uniform(minx, maxx)
    y = np.random.uniform(miny, maxy)
    pt = Point(x,y)
    w0 = pt.within(borders0)
    w1 = pt.within(borders1)
    if w0 and not w1:
        hits0only.append([x, y])
    if w1 and not w0:
        hits1only.append([x, y])

In [ ]:
def pois_to_list(d):
    dat =[]
    for p in d['results']:
        if 'synagogue' in p['types']:
            datum = {}
            for f in ['name', 'place_id', 'vicinity']:
                datum[f] = p[f]
            for f in ['lat', 'lng']:
                datum[f] = p['geometry']['location'][f]
            dat.append(datum)
    return dat

In [ ]:
dat = []
infile='data/pois/Haifa.json'
with open(infile) as json_file:
      d=json.load(json_file)
      dat.extend(pois_to_list(d))

In [ ]:
d2 = pd.DataFrame(dat)
d2

In [ ]:
set([1, 2, 3, 4]) - set([2])

In [ ]:
pt = (29.751355, 34.373654) # Al temd
pt = (29.968069, 34.778875 ) # somewhere in Eilot
pt = (30.184895718556003, 34.66956055958991) # somewhere in Egypt
pt = (30.79287035913899, 34.47324188156933) # ezuz
border_nw = (31.332297683286306, 34.236524779286576)   # northest point of the gaza/egypt border
border_se = ( 29.5224088182224, 34.92718133237316 )  # Taba crossing

xA = pt[1]
yA = pt[0]
x1 = border_nw[1]
y1 = border_nw[0]
x2 = border_se[1]
y2 = border_se[0]


v1 = (x2-x1, y2-y1)   # Vector 1
v2 = (x2-xA, y2-yA)   # Vector 2
xp = v1[0]*v2[1] - v1[1]*v2[0]  # Cross product
if xp > 0:
    print('below the line')
elif xp < 0:
    print('above the line')
else:
    print('on the same line!')


In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat='lat', lon='lng', center=dict(lat=32, lon=34.8), zoom=7, mapbox_style="stamen-terrain")
fig.update_traces(marker={'size': 9})
fig.show()